In [2]:
using DeferredAcceptance

┌ Info: Precompiling DeferredAcceptance [7ba42312-65e6-11eb-2a40-f37f14131e13]
└ @ Base loading.jl:1317


Most school-choice algorithms start with each side ranking the other
in terms of preferability. First we input the student preferences.
Each column is a student, and each row is a school. So, the 2 in cell
(3, 4) means that student 4 has named school 3 as her 2nd choice.

In [7]:
students = [3 3 4 3 4 3 3 3 3 4;
            4 4 3 4 3 4 4 4 4 3;
            2 1 2 2 2 1 2 2 2 2;
            1 2 1 1 1 2 1 1 1 1]

4×10 Matrix{Int64}:
 3  3  4  3  4  3  3  3  3  4
 4  4  3  4  3  4  4  4  4  3
 2  1  2  2  2  1  2  2  2  2
 1  2  1  1  1  2  1  1  1  1

Now we input the school preferences over the students. Notice that the
shape is transposed: each column is a school, and row a student. This
format improves computation speed. Also notice that now we have ties:
for example, school 1 has four students tied for 1st place. You can
also write the first column as `[1, 1, 2, 2, 4, 1, 2, 3, 3, 1]`. Basically,
they just need to be positive integers in descending order of preference. 

In [8]:
schools = [1 5 7 5;
           1 1 1 1;
           5 1 1 1;
           5 1 1 1;
           10 9 7 8;
           1 5 4 5;
           5 1 4 1;
           8 5 7 8;
           8 9 7 8;
           1 5 4 5]

10×4 Matrix{Int64}:
  1  5  7  5
  1  1  1  1
  5  1  1  1
  5  1  1  1
 10  9  7  8
  1  5  4  5
  5  1  4  1
  8  5  7  8
  8  9  7  8
  1  5  4  5

Finally we have the capacities, or number of students each school can accept.

In [9]:
capacities = [3, 2, 2, 3]

4-element Vector{Int64}:
 3
 2
 2
 3

Let's use the DA algorithm to find a stable match. The DA algorithm requires
that both students and schools have strict preferences, so we need to run
some random tiebreaking to make the school preferences strict. We will use
STB, which is usually the best choice.       

In [10]:
schools_tiebroken = singletiebreaking(schools)

10×4 Matrix{Int64}:
  1   5   7   5
  2   2   2   2
  5   1   1   1
  6   3   3   3
 10  10  10  10
  3   7   4   6
  7   4   6   4
  8   6   8   8
  9   9   9   9
  4   8   5   7

Now we run DA.

In [11]:
assn, ranks = deferredacceptance(students, schools_tiebroken, capacities; verbose=true)

Round 1
  School 4 rejects student 1
  School 4 rejects student 10
  School 4 rejects student 8
  School 4 rejects student 9
  School 4 rejects student 5
Round 2
  School 3 rejects student 10
  School 3 rejects student 1
  School 3 rejects student 8
  School 3 rejects student 9
  School 3 rejects student 5
Round 3
DA terminated in 3 iterations


([1, 3, 4, 4, 2, 3, 4, 1, 1, 2], [3, 1, 1, 1, 3, 1, 1, 3, 3, 3])

Make sure the match is stable.

In [12]:
@assert isstable(students, schools, capacities, assn)

The first output is the assignment. Student 1 goes to school 1, student 2
goes to school 3, etc. If the market had too many students, we would have
some students assigned to school 5, which means no school accepted them.

In [14]:
println(assn)

[1, 3, 4, 4, 2, 3, 4, 1, 1, 2]


The second output is the rank each student gave to their assigned school.
So student 1 got her 3rd choice, student 2 got her 2nd, etc.   

In [15]:
println(ranks)

[3, 1, 1, 1, 3, 1, 1, 3, 3, 3]


A common measure of student disutility:

In [16]:
println(sum(ranks))

20


Since the tiebreaking mechanism involves randomness, you might have
found a slightly different assignment.

Now let's try something different. Suppose the schools don't have any
preferences over the students at all (as in New Orleans). Then we can
start with an arbitrary assignment and exchange Pareto-improving pairs
to get a better one. This is called top trading cycles and is a classic
solution to the "housing market" problem. In school choice, it produces
an assignment that tends to maximize student welfare.      

In [17]:
assn, ranks = TTC_match(students, capacities; verbose=true)

Searching for Pareto-improving cycles at rank 1
Current assignment: [4, 1, 2, 1, 2, 1, 4, 3, 4, 3]
  Swap requests: Dict(5 => 1, 4 => 1, 6 => 8, 7 => 7, 2 => 8, 10 => 1, 9 => 9, 8 => 9, 3 => 7, 1 => 7)
  Cycle involving students [7]
  Cycle involving students [9]
Searching for Pareto-improving cycles at rank 2
Current assignment: [4, 1, 2, 1, 2, 1, 4, 3, 4, 3]
  Swap requests: Dict(5 => 8, 4 => 8, 6 => 7, 2 => 9, 10 => 8, 8 => 10, 3 => 10, 1 => 10)
  Cycle involving students [8, 10]
  Cycle involving students [10, 8]
Searching for Pareto-improving cycles at rank 3
Current assignment: [4, 1, 2, 1, 2, 1, 4, 3, 4, 3]
  Swap requests: Dict(5 => 5, 4 => 6, 6 => 2, 2 => 2, 3 => 5, 1 => 4)
  Cycle involving students [5]
  Cycle involving students [2]


([4, 1, 2, 1, 2, 1, 4, 3, 4, 3], [1, 3, 3, 3, 3, 3, 1, 2, 1, 2])

In [18]:
# Same output format
println(assn)

[4, 1, 2, 1, 2, 1, 4, 3, 4, 3]


In [19]:
println(ranks)

[1, 3, 3, 3, 3, 3, 1, 2, 1, 2]


In [20]:
# Student disutility
println(sum(ranks))
# 21

22


Looks like TTC did a little better, but what are the tradeoffs?

In [21]:
isstable(students, schools, capacities, assn; verbose=true)

Student feas. :  true
School feas.  :  true
Stability     :  false


false

### Nonatomic model

Let's try something different: the nonatomic DA model. Under this model, we
have a measure (usually a percentage) of students associated with each preference
list. Instead of admitting individual students, stable matches can be characterized
by score cutoffs at each school.

In [24]:
# Possible preference lists: Most students have (1, 2, 3, 4), but some prefer 2 to 1.
students = [1 2;
            2 1;
            3 3;
            4 4]

# Percentage of students with each preference list.
students_dist = [0.75, 0.25]

#=  School capacities. Schools have no preference list; assume student preferability is
    independently distributed in each group.        =#
capacities = [0.2, 0.2, 0.2, 0.2]

# Run nonatomic DA.
assn, rdist, cutoffs = nonatomicdeferredacceptance(students, students_dist, nothing, capacities;
                                                   verbose=true, return_cutoffs=true)

# Equivalently,
# cutoffs = nonatomicdeferredacceptance_iid(students, students_dist, capacities)

# As expected, school 1 is most selective.
println(cutoffs)
# [0.7873499783936067, 0.7620499351812947, 0.666666666665277, 0.49999999999882583]


Round 1
  Demand at school 1 was 0.75 > capacity 0.2
    Old cutoff:  0.0
    New cutoff:  0.7333333333333334
  Demand at school 2 was 0.25 > capacity 0.2
    Old cutoff:  0.0
    New cutoff:  0.19999999999999996

Round 2
  Demand at school 1 was 0.2133333333333333 > capacity 0.2
    Old cutoff:  0.7333333333333334
    New cutoff:  0.75
  Demand at school 2 was 0.6400000000000001 > capacity 0.2
    Old cutoff:  0.19999999999999996
    New cutoff:  0.75

Round 3
  Demand at school 1 was 0.234375 > capacity 0.2
    Old cutoff:  0.75
    New cutoff:  0.7866666666666666
  Demand at school 2 was 0.203125 > capacity 0.2
    Old cutoff:  0.75
    New cutoff:  0.7538461538461538
  Demand at school 3 was 0.5625 > capacity 0.2
    Old cutoff:  0.0
    New cutoff:  0.6444444444444444

Round 4
  Demand at school 1 was 0.20020512820512823 > capacity 0.2
    Old cutoff:  0.7866666666666666
    New cutoff:  0.7868852459016393
  Demand at school 2 was 0.20676923076923076 > capacity 0.2
    Old cutoff